In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import cv2, os, shutil, re, math
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.layers import *

In [ ]:
df = pd.DataFrame()
filepaths = []
labels = []

train_filepath = [i for i in os.listdir('/kaggle/input/fishes-species-in-the-indian-subcontinent/StockFish-1/train') if i.endswith('.jpg')]
train_filepaths = [os.path.join('/kaggle/input/fishes-species-in-the-indian-subcontinent/StockFish-1/train/', i) for i in train_filepath]
train_label = [i.split('_')[0] for i in train_filepath]
for i in range(len(train_filepaths)):
    filepaths.append(train_filepaths[i])
    labels.append(train_label[i])

test_filepath = [i for i in os.listdir('/kaggle/input/fishes-species-in-the-indian-subcontinent/StockFish-1/test') if i.endswith('.jpg')]
test_filepaths = [os.path.join('/kaggle/input/fishes-species-in-the-indian-subcontinent/StockFish-1/test/', i) for i in test_filepath]
test_label = [i.split('_')[0] for i in test_filepath]
for i in range(len(test_filepaths)):
    filepaths.append(test_filepaths[i])
    labels.append(test_label[i])

valid_filepath = [i for i in os.listdir('/kaggle/input/fishes-species-in-the-indian-subcontinent/StockFish-1/valid') if i.endswith('.jpg')]
valid_filepaths = [os.path.join('/kaggle/input/fishes-species-in-the-indian-subcontinent/StockFish-1/valid/', i) for i in valid_filepath]
valid_label = [i.split('_')[0] for i in valid_filepath]
for i in range(len(valid_filepaths)):
    filepaths.append(valid_filepaths[i])
    labels.append(valid_label[i])


df['filepaths'] = filepaths
df['labels'] = labels

In [ ]:
df

In [ ]:
def create_data(df):
    train_df, dummy_df = train_test_split(df, train_size=.8, shuffle=True, random_state=123, stratify=df['labels'])
    valid_df, test_df = train_test_split(dummy_df, train_size=.5, shuffle=True, random_state=123,
                                         stratify=dummy_df['labels'])
    classes = sorted(train_df['labels'].unique())
    class_count = len(classes)
    sample_df = train_df.sample(n=50, replace=False)
    ht = 0
    wt = 0
    count = 0
    for i in range(len(sample_df)):
        fpath = sample_df['filepaths'].iloc[i]
        try:
            img = cv2.imread(fpath)
            h = img.shape[0]
            w = img.shape[1]
            wt += w
            ht += h
            count += 1
        except:
            pass
    have = int(ht / count)
    wave = int(wt / count)
    aspect_ratio = have / wave
    print('number of classes in processed dataset= ', class_count)
    counts = list(train_df['labels'].value_counts())
    print('the maximum files in any class in train_df is ', max(counts),
          '  the minimum files in any class in train_df is ', min(counts))
    print('train_df length: ', len(train_df), '  test_df length: ', len(test_df), '  valid_df length: ', len(valid_df))
    print('average image height= ', have, '  average image width= ', wave, ' aspect ratio h/w= ', aspect_ratio)
    
    return train_df, test_df, valid_df, classes, class_count

train_df, test_df, valid_df, classes, class_count = create_data(df)

In [ ]:
epochs = 30
batch_size = 32
img_size=(300, 300)
input_shape = (300, 300, 3)

In [ ]:
def make_gens(batch_size, train_df, test_df, valid_df, img_size):
    trgen = ImageDataGenerator(horizontal_flip=True)
    t_and_v_gen = ImageDataGenerator()
    msg = '{0:70s} for train generator'.format(' ')
    print(msg, '\r', end='')  # prints over on the same line
    train_ds = trgen.flow_from_dataframe(train_df, x_col='filepaths', y_col='labels',
                                         target_size=img_size, class_mode='categorical',
                                         color_mode='rgb', batch_size=batch_size, shuffle=True)

    msg = '{0:70s} for valid generator'.format(' ')
    print(msg, '\r', end='')  # prints over on the same line
    valid_ds = t_and_v_gen.flow_from_dataframe(valid_df, x_col='filepaths', y_col='labels',
                                         target_size=img_size, class_mode='categorical',
                                         color_mode='rgb', batch_size=batch_size, shuffle=False)

    test_len = len(test_df)
    test_batch_size = sorted([int(test_len / n) for n in range(1, test_len + 1)
                              if test_len % n == 0 and test_len / n<=80], reverse=True)[0]
    test_steps = int(test_len / test_batch_size)
    msg = '{0:70s} for test generator'.format(' ')
    print(msg, '\r', end='')  # prints over on the same line
    test_ds = t_and_v_gen.flow_from_dataframe(test_df, x_col='filepaths', y_col='labels',
                                               target_size=img_size, class_mode='categorical',
                                               color_mode='rgb', batch_size=batch_size, shuffle=False)

    classes = list(train_ds.class_indices.keys())
    # class_indices = list(train_ds.class_indices.values())
    class_count = len(classes)
    # labels = test_ds.labels
    print('test batch size: ', test_batch_size, 'test steps: ', test_steps, 'number of classes : ', class_count)

    # return train_ds, test_ds, valid_ds, test_batch_size, test_steps, classes

    return train_ds, test_ds, valid_ds


train_ds, test_ds, valid_ds = make_gens(batch_size, train_df, test_df, valid_df, img_size)

In [ ]:
def GELU(x):
    res = 0.5 * x * (1 + tf.nn.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * (x ** 3))))
    return res


class ResMLPBlock(tf.keras.layers.Layer):
    def __init__(self, units, residual_path):
        super(ResMLPBlock, self).__init__()
        self.residual_path = residual_path
        self.D1 = Dense(units, activation='relu')
        self.D2 = Dense(units, activation='relu')

        if self.residual_path:
            self.D3 = Dense(units)
            self.D4 = Dense(units)

    def call(self, inputs):
        residual = inputs

        x = self.D1(inputs)
        y = self.D2(x)

        if self.residual_path:
            residual = self.D3(inputs)
            residual = GELU(residual)
            residual = self.D4(residual)
            residual = GELU(residual)

        output = y + residual
        return output



class ResMLP(tf.keras.layers.Layer):
    def __init__(self, initial_filters, block_list, num_classes):
        super(ResMLP, self).__init__()
        self.initial_filters = initial_filters
        self.block_list = block_list

        self.D1 = Dense(self.initial_filters, activation='relu')
        self.B1 = BatchNormalization()

        self.blocks = tf.keras.models.Sequential()
        for block_id in range(len(block_list)):
            for layer_id in range(block_list[block_id]):
                if block_id != 0 and layer_id == 0:
                    block = ResMLPBlock(units=self.initial_filters, residual_path=True)
                else:
                    block = ResMLPBlock(units=self.initial_filters, residual_path=False)
                self.blocks.add(block)
            self.initial_filters *= 2

        self.D2 = Dense(num_classes, activation='softmax')


    def call(self, inputs):
        x = self.D1(inputs)
        x = self.B1(x)
        x = self.blocks(x)
        y = self.D2(x)
        return y

In [ ]:
class BottleNeck(tf.keras.layers.Layer):
    def __init__(self, filter_num, stride=1):
        super(BottleNeck, self).__init__()
        self.C1 = Conv2D(filters=filter_num, kernel_size=(1, 1), strides=1, padding='same')
        self.B1 = BatchNormalization()
        self.A1 = Activation('relu')

        self.C2 = Conv2D(filters=filter_num, kernel_size=(3, 3), strides=stride, padding='same')
        self.B2 = BatchNormalization()
        self.A2 = Activation('relu')

        self.C3 = Conv2D(filters=filter_num * 4, kernel_size=(1, 1), strides=1, padding='same')
        self.B3 = BatchNormalization()

        self.downsample = tf.keras.models.Sequential()
        self.downsample.add(Conv2D(filters=filter_num * 4, kernel_size=(1, 1), strides=stride))
        self.downsample.add(BatchNormalization())

    def call(self, inputs, training=None):
        residual = self.downsample(inputs)

        x = self.C1(inputs)
        x = self.B1(x, training=training)
        x = self.A1(x)

        x = self.C2(x)
        x = self.B2(x, training=training)
        x = self.A2(x)

        x = self.C3(x)
        x = self.B3(x, training=training)
        output = tf.nn.relu(tf.keras.layers.add([residual, x]))

        return output

    
def make_bottleneck_layer(filter_num, blocks, stride=1):
    res_block = tf.keras.Sequential()
    res_block.add(BottleNeck(filter_num, stride=stride))

    for _ in range(1, blocks):
        res_block.add(BottleNeck(filter_num, stride=1))

    return res_block

In [ ]:
class ResNetTypeII(tf.keras.layers.Layer):
    def __init__(self, layer_params, input_shape):
        super(ResNetTypeII, self).__init__()
        assert len(layer_params) == 4, 'The length of layer_params must == 4!!!'

        self.C1 = Conv2D(filters=64, kernel_size=(7, 7), strides=2, padding='same', input_shape=input_shape)
        self.B1 = BatchNormalization()
        self.A1 = Activation('relu')
        self.P1 = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')

        self.layer1 = make_bottleneck_layer(filter_num=64,
                                             blocks=layer_params[0])
        self.layer2 = make_bottleneck_layer(filter_num=128,
                                             blocks=layer_params[1],
                                             stride=2)
        self.layer3 = make_bottleneck_layer(filter_num=256,
                                             blocks=layer_params[2],
                                             stride=2)
        self.layer4 = make_bottleneck_layer(filter_num=512,
                                             blocks=layer_params[3],
                                             stride=2)
        self.P2 = GlobalAveragePooling2D()
        # self.D1 = Dense(units=NUM_CLASSES, activation='softmax')

    def call(self, inputs, training=None):
        x = self.C1(inputs)
        x = self.B1(x, training=training)
        x = self.A1(x)
        x = self.P1(x)

        x = self.layer1(x, training=training)
        x = self.layer2(x, training=training)
        x = self.layer3(x, training=training)
        x = self.layer4(x, training=training)
        y = self.P2(x)
        # y = self.D1(x)

        return y

In [ ]:
class NeuralNetWork(tf.keras.Model):
    def __init__(self, layer_params, initial_units, block_list, num_classes):
        super(NeuralNetWork, self).__init__()

        self.layer1 = ResNetTypeII(layer_params=layer_params, input_shape=input_shape)

        self.layer2 = ResMLP(initial_filters=initial_units, block_list=block_list, num_classes=num_classes)


    def call(self, x):
        x = self.layer1(x)
        y = self.layer2(x)
        return y

In [ ]:
net = NeuralNetWork(layer_params=[3, 4, 6, 3],  initial_units=64, block_list=[2, 2, 2], num_classes=5)

net.compile(optimizer='adam',
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
            metrics=['categorical_accuracy', 'AUC'])

checkpoint_save_path = './Model.ckpt'
if os.path.exists(checkpoint_save_path + '.index'):
    net.load_weights(checkpoint_save_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                 save_weights_only=True,
                                                 save_best_only=True)

history = net.fit(train_ds, epochs=epochs, batch_size=batch_size, callbacks=[cp_callback])

net.summary()

file = open('./weights.txt', 'w')
for v in net.trainable_variables:
    file.write(str(v.name) + '\n')
    file.write(str(v.shape) + '\n')
    file.write(str(v.numpy()) + '\n')

file.close()

In [ ]:
acc = history.history['categorical_accuracy']
auc = history.history['auc']
loss = history.history['loss']


plt.figure(figsize=(12, 12))
plt.plot(acc, label='Training ACC')
plt.plot(auc, label='Training AUC')
plt.plot(loss, label='Training LOSS')
plt.title('Training ACC AUC LOSS')
plt.legend()

plt.show()